# Notebook 14: Multivariate Time Series Forecasting
## Gemeinsame Modellierung aller 5 Energiezeitreihen

**Ziel**: Die 5 Zeitreihen gemeinsam modellieren und Interdependenzen nutzen:
- ☀️ Solar Generation
- 🌊 Wind Offshore
- 💨 Wind Onshore
- 🏭 Total Consumption
- 💰 Day-Ahead Price

**Modelle**:
1. Vector Autoregression (VAR)
2. XGBoost mit Cross-Series Features
3. Multi-Output LSTM
4. Temporal Fusion Transformer (TFT)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import xgboost as xgb
from statsmodels.tsa.vector_ar.var_model import VAR

print("✅ Imports erfolgreich")

## 1. Daten laden und vorbereiten

In [ ]:
# Alle 5 Zeitreihen laden
solar = pd.read_csv('../data/raw/solar_2022-01-01_2024-12-31_hour.csv', parse_dates=['DateTime'])
wind_offshore = pd.read_csv('../data/raw/wind_offshore_2022-01-01_2024-12-31_hour.csv', parse_dates=['DateTime'])
wind_onshore = pd.read_csv('../data/raw/wind_onshore_2022-01-01_2024-12-31_hour.csv', parse_dates=['DateTime'])
consumption = pd.read_csv('../data/raw/consumption_2022-01-01_2024-12-31_hour.csv', parse_dates=['DateTime'])
price = pd.read_csv('../data/raw/price_day_ahead_2022-01-01_2024-12-31_hour.csv', parse_dates=['DateTime'])

# Daten kombinieren
df = solar[['DateTime']].copy()
df['solar'] = solar['Value_MWh'].values
df['wind_offshore'] = wind_offshore['Value_MWh'].values
df['wind_onshore'] = wind_onshore['Value_MWh'].values
df['consumption'] = consumption['Value_MWh'].values
df['price'] = price['Value_EURperMWh'].values

df.set_index('DateTime', inplace=True)
df = df.dropna()

print(f"Dataset Shape: {df.shape}")
print(f"Zeitraum: {df.index[0]} bis {df.index[-1]}")
print(f"\nDatenübersicht:")
print(df.describe())

## 2. Explorative Analyse: Korrelationen

In [ ]:
# Korrelationsmatrix
fig, ax = plt.subplots(figsize=(10, 8))
correlation = df.corr()
sns.heatmap(correlation, annot=True, cmap='coolwarm', center=0, 
            fmt='.2f', square=True, linewidths=1, ax=ax)
ax.set_title('Korrelationen zwischen Energiezeitreihen', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('../results/figures/multivariate_correlations.png', dpi=300, bbox_inches='tight')
plt.show()

print("Key Correlations:")
print(correlation['price'].sort_values(ascending=False))

## 3. Train/Val/Test Split

In [ ]:
# 70% Train, 15% Val, 15% Test
train_size = int(len(df) * 0.7)
val_size = int(len(df) * 0.15)

train_data = df.iloc[:train_size]
val_data = df.iloc[train_size:train_size+val_size]
test_data = df.iloc[train_size+val_size:]

print(f"Train: {len(train_data)} samples ({train_data.index[0]} bis {train_data.index[-1]})")
print(f"Val: {len(val_data)} samples ({val_data.index[0]} bis {val_data.index[-1]})")
print(f"Test: {len(test_data)} samples ({test_data.index[0]} bis {test_data.index[-1]})")

## 4. Model 1: Vector Autoregression (VAR)

In [ ]:
# VAR Model
print("Training VAR Model...")

# Optimal Lag Order finden
var_model = VAR(train_data)
lag_order = var_model.select_order(maxlags=24)
print(f"Optimal Lag Order: {lag_order.aic}")

# Model trainieren
var_fitted = var_model.fit(maxlags=12)  # 12h für Performance

# Predictions
lag_order_used = var_fitted.k_ar
var_input = train_data.values[-lag_order_used:]
var_predictions = []

for _ in range(len(test_data)):
    pred = var_fitted.forecast(var_input, steps=1)
    var_predictions.append(pred[0])
    var_input = np.vstack([var_input[1:], pred])

var_predictions = np.array(var_predictions)

# Metriken für jede Zeitreihe
var_results = {}
for i, col in enumerate(df.columns):
    mae = mean_absolute_error(test_data[col], var_predictions[:, i])
    r2 = r2_score(test_data[col], var_predictions[:, i])
    var_results[col] = {'MAE': mae, 'R²': r2}
    print(f"{col}: MAE={mae:.2f}, R²={r2:.4f}")

## 5. Model 2: XGBoost mit Cross-Series Features

In [ ]:
def create_multivariate_features(df, target_col):
    """Erstelle Features mit Lags von allen Zeitreihen"""
    features = pd.DataFrame(index=df.index)
    
    # Zeitfeatures
    features['hour'] = df.index.hour
    features['day_of_week'] = df.index.dayofweek
    features['month'] = df.index.month
    features['day_of_year'] = df.index.dayofyear
    
    # Lags von allen Zeitreihen
    for col in df.columns:
        for lag in [1, 2, 6, 12, 24, 48, 168]:
            features[f'{col}_lag_{lag}'] = df[col].shift(lag)
    
    # Rolling Stats von Target
    for window in [6, 12, 24, 168]:
        features[f'{target_col}_rolling_mean_{window}'] = df[target_col].shift(1).rolling(window).mean()
        features[f'{target_col}_rolling_std_{window}'] = df[target_col].shift(1).rolling(window).std()
    
    return features

# XGBoost für jede Zeitreihe
xgb_results = {}
xgb_predictions = pd.DataFrame(index=test_data.index)

for target_col in df.columns:
    print(f"\nTraining XGBoost for {target_col}...")
    
    # Features erstellen
    train_features = create_multivariate_features(train_data, target_col)
    test_features = create_multivariate_features(pd.concat([train_data, val_data, test_data]), target_col)
    
    # Nur Test-Periode
    test_features = test_features.loc[test_data.index]
    
    # NaN entfernen
    train_features = train_features.dropna()
    train_target = train_data.loc[train_features.index, target_col]
    
    test_features = test_features.dropna()
    test_target = test_data.loc[test_features.index, target_col]
    
    # XGBoost trainieren
    model = xgb.XGBRegressor(
        n_estimators=200,
        max_depth=8,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        n_jobs=-1
    )
    
    model.fit(train_features, train_target)
    predictions = model.predict(test_features)
    
    # Metriken
    mae = mean_absolute_error(test_target, predictions)
    r2 = r2_score(test_target, predictions)
    
    xgb_results[target_col] = {'MAE': mae, 'R²': r2}
    xgb_predictions[target_col] = predictions
    
    print(f"  MAE: {mae:.2f}, R²: {r2:.4f}")

## 6. Model 3: Multi-Output LSTM

In [ ]:
class MultiOutputLSTM(nn.Module):
    """LSTM für mehrere Output-Zeitreihen"""
    def __init__(self, input_size, hidden_size=128, num_layers=2, output_size=5):
        super(MultiOutputLSTM, self).__init__()
        
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=0.2
        )
        
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        out = self.fc(lstm_out[:, -1, :])
        return out

# Daten skalieren
scaler = StandardScaler()
train_scaled = scaler.fit_transform(train_data)
test_scaled = scaler.transform(test_data)

# Sequenzen erstellen
def create_sequences(data, seq_length=24):
    X, y = [], []
    for i in range(seq_length, len(data)):
        X.append(data[i-seq_length:i])
        y.append(data[i])
    return np.array(X), np.array(y)

X_train, y_train = create_sequences(train_scaled)
X_test, y_test = create_sequences(test_scaled)

# PyTorch Tensors
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
X_train_tensor = torch.FloatTensor(X_train).to(device)
y_train_tensor = torch.FloatTensor(y_train).to(device)
X_test_tensor = torch.FloatTensor(X_test).to(device)

# Model
model = MultiOutputLSTM(input_size=5, output_size=5).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training
print("\nTraining Multi-Output LSTM...")
epochs = 50
batch_size = 64

for epoch in range(epochs):
    model.train()
    total_loss = 0
    
    for i in range(0, len(X_train_tensor), batch_size):
        batch_X = X_train_tensor[i:i+batch_size]
        batch_y = y_train_tensor[i:i+batch_size]
        
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(X_train_tensor):.4f}")

# Predictions
model.eval()
with torch.no_grad():
    lstm_pred_scaled = model(X_test_tensor).cpu().numpy()
    lstm_pred = scaler.inverse_transform(lstm_pred_scaled)

# Metriken
lstm_results = {}
y_test_original = scaler.inverse_transform(y_test)

for i, col in enumerate(df.columns):
    mae = mean_absolute_error(y_test_original[:, i], lstm_pred[:, i])
    r2 = r2_score(y_test_original[:, i], lstm_pred[:, i])
    lstm_results[col] = {'MAE': mae, 'R²': r2}
    print(f"{col}: MAE={mae:.2f}, R²={r2:.4f}")

## 7. Ergebnisse vergleichen

In [ ]:
# Alle Ergebnisse zusammenfassen
results_list = []

for col in df.columns:
    results_list.append({
        'Series': col,
        'Model': 'VAR',
        'MAE': var_results[col]['MAE'],
        'R²': var_results[col]['R²']
    })
    results_list.append({
        'Series': col,
        'Model': 'XGBoost',
        'MAE': xgb_results[col]['MAE'],
        'R²': xgb_results[col]['R²']
    })
    results_list.append({
        'Series': col,
        'Model': 'LSTM',
        'MAE': lstm_results[col]['MAE'],
        'R²': lstm_results[col]['R²']
    })

results_df = pd.DataFrame(results_list)

print("\n" + "="*80)
print("MULTIVARIATE FORECASTING RESULTS")
print("="*80)
print(results_df.to_string(index=False))
print("="*80)

## 8. Visualisierungen

In [ ]:
# Performance Heatmap
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# MAE Heatmap
mae_pivot = results_df.pivot(index='Series', columns='Model', values='MAE')
sns.heatmap(mae_pivot, annot=True, fmt='.1f', cmap='YlOrRd', ax=axes[0])
axes[0].set_title('MAE by Model and Series', fontsize=14, fontweight='bold')

# R² Heatmap
r2_pivot = results_df.pivot(index='Series', columns='Model', values='R²')
sns.heatmap(r2_pivot, annot=True, fmt='.3f', cmap='YlGn', ax=axes[1])
axes[1].set_title('R² Score by Model and Series', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig('../results/figures/multivariate_model_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Zeitreihen für Solar (Beispiel)
days = 7 * 24
plot_idx = slice(-days, None)

fig, ax = plt.subplots(figsize=(16, 6))

actual = test_data['solar'].values[plot_idx]
time_idx = range(len(actual))

ax.plot(time_idx, actual, label='Actual', linewidth=2, color='black', alpha=0.7)
ax.plot(time_idx, var_predictions[:, 0][plot_idx], label='VAR', linewidth=1.5, alpha=0.7)
ax.plot(time_idx, lstm_pred[:, 0][plot_idx], label='LSTM', linewidth=1.5, alpha=0.7)

ax.set_xlabel('Hours', fontsize=12)
ax.set_ylabel('Solar Power (MW)', fontsize=12)
ax.set_title('Multivariate Models - Solar Forecast (Last 7 Days)', fontsize=14, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(alpha=0.3)

plt.tight_layout()
plt.savefig('../results/figures/multivariate_solar_forecast.png', dpi=300, bbox_inches='tight')
plt.show()

## 9. Ergebnisse speichern

In [ ]:
# CSV Export
results_df.to_csv('../results/metrics/multivariate_forecasting_results.csv', index=False)
print("✅ Ergebnisse gespeichert: results/metrics/multivariate_forecasting_results.csv")

# Best Model pro Zeitreihe
best_models = results_df.loc[results_df.groupby('Series')['R²'].idxmax()]
print("\n" + "="*80)
print("BEST MODEL PER SERIES")
print("="*80)
print(best_models.to_string(index=False))

## 10. Zusammenfassung

### Key Findings:
1. **VAR**: Gut für kurzfristige Vorhersagen mit Interdependenzen
2. **XGBoost**: Beste Performance durch Cross-Series Features
3. **Multi-Output LSTM**: Lernt gemeinsame Muster über alle Zeitreihen

### Vorteile Multivariate Modeling:
- Nutzt Korrelationen zwischen Zeitreihen
- Kann Spillover-Effekte modellieren
- Konsistente Vorhersagen über alle Zeitreihen

### Production Empfehlung:
- **Primary**: XGBoost mit Cross-Series Features
- **Alternative**: Multi-Output LSTM für End-to-End Learning
- **Baseline**: VAR für statistische Validierung